### Imports

In [1]:
from tqdm import tqdm
from utils.utils import *

import networkx as nx
import numpy as np
import pandas as pd
import os
import sys

%load_ext autotime

/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/li

time: 160 µs (started: 2023-07-11 10:49:30 -05:00)


### Parameters

In [2]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

dir = os.path.dirname(os.getcwd())
print(dir)

exp = params["exp"]
print("Exp:\t\t", exp)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

option = params["option"]
print("Option:\t", option)

/home/ealvarez/Project/GNN_Filter
Exp:		 exp8
Groups id:	 ['Starved', 'FCSglc', 'DMA']
Subgroups id:	 {'Starved': ['1', '2'], 'FCSglc': ['1', '2'], 'DMA': ['1', '2']}
Option:	 
time: 849 µs (started: 2023-07-11 10:49:30 -05:00)


### Load dataset

In [3]:
# load dataset groups
df_join_raw = pd.read_csv("input/{}_raw.csv".format(exp), index_col=0)
df_join_raw

,Starved1.1,Starved1.2,Starved2.1,Starved2.2,FCSglc1.1,FCSglc1.2,FCSglc2.1,FCSglc2.2,DMA1.1,DMA1.2,DMA2.1,DMA2.2
59.01425,0.000310,0.000250,0.000540,0.000470,0.000300,0.000310,0.000240,0.000220,0.000250,0.000230,0.000240,0.000190
61.98612,0.000220,0.000240,0.000230,0.000270,0.000150,0.000170,0.000170,0.000190,0.000160,0.000140,0.000180,0.000170
71.01184,0.000042,0.000053,0.000042,0.000036,0.000130,0.000160,0.000110,0.000150,0.000047,0.000094,0.000130,0.000100
74.00508,0.000015,0.000021,0.000008,0.000013,0.000800,0.000910,0.000590,0.000670,0.001010,0.000940,0.000750,0.001160
74.02387,0.000034,0.000039,0.000043,0.000044,0.000130,0.000043,0.000110,0.000100,0.000039,0.000110,0.000059,0.000040
...,...,...,...,...,...,...,...,...,...,...,...,...
1177.92834,0.000010,0.000010,0.000009,0.000009,0.000006,0.000011,0.000006,0.000004,0.000018,0.000005,0.000011,0.000092
1178.40808,0.000010,0.000005,0.000006,0.000005,0.000014,0.000028,0.000034,0.000027,0.000078,0.000017,0.000085,0.000005
1180.43262,0.000002,0.000002,0.000002,0.000002,0.001720,0.001510,0.001630,0.001570,0.001710,0.000480,0.001670,0.001080
1185.20862,0.000007,0.000270,0.000300,0.000330,0.000007,0.000045,0.000030,0.000028,0.000005,0.000016,0.000035,0.000022


time: 22.4 ms (started: 2023-07-11 10:49:30 -05:00)


### Generate graphs

In [4]:
# logarithm
df_join_raw_log = log10_global(df_join_raw)
df_join_raw_log.head()

,Starved1.1,Starved1.2,Starved2.1,Starved2.2,FCSglc1.1,FCSglc1.2,FCSglc2.1,FCSglc2.2,DMA1.1,DMA1.2,DMA2.1,DMA2.2
59.01425,-3.508638,-3.602060,-3.267606,-3.327902,-3.522879,-3.508638,-3.619789,-3.657577,-3.602060,-3.638272,-3.619789,-3.721246
61.98612,-3.657577,-3.619789,-3.638272,-3.568636,-3.823909,-3.769551,-3.769551,-3.721246,-3.795880,-3.853872,-3.744727,-3.769551
71.01184,-4.379864,-4.279841,-4.373660,-4.447332,-3.886057,-3.795880,-3.958607,-3.823909,-4.326058,-4.028724,-3.886057,-4.000000
74.00508,-4.838632,-4.679854,-5.089909,-4.882729,-3.096910,-3.040959,-3.229148,-3.173925,-2.995679,-3.026872,-3.124939,-2.935542
74.02387,-4.471083,-4.414539,-4.364516,-4.353596,-3.886057,-4.369572,-3.958607,-4.000000,-4.413413,-3.958607,-4.232102,-4.402305


time: 13.1 ms (started: 2023-07-11 10:49:30 -05:00)


In [5]:
# split graph in groups and subgroups

list_df_groups_subgroups = split_groups_subgroups(df_join_raw_log, groups_id, subgroups_id)
list_df_groups_subgroups[2][1].head()

,DMA2.1,DMA2.2
59.01425,-3.619789,-3.721246
61.98612,-3.744727,-3.769551
71.01184,-3.886057,-4.000000
74.00508,-3.124939,-2.935542
74.02387,-4.232102,-4.402305


time: 6.76 ms (started: 2023-07-11 10:49:30 -05:00)


In [6]:
# transpose
list_groups_subgroups_t = transpose_global(list_df_groups_subgroups)
list_groups_subgroups_t[2][1].head()

,59.01425,61.98612,71.01184,74.00508,74.02387,78.95773,82.02924,83.06079,84.00671,84.04674,...,1155.95496,1158.45068,1166.35645,1166.93823,1170.16260,1177.92834,1178.40808,1180.43262,1185.20862,1197.05481
0,-3.619789,-3.744727,-3.886057,-3.124939,-4.232102,-4.216096,-4.519993,-3.853872,-4.111821,-3.537602,...,-3.823909,-3.537602,-4.721246,-3.958607,-5.341989,-4.962574,-4.071092,-2.777284,-4.450997,-4.534617
1,-3.721246,-3.769551,-4.000000,-2.935542,-4.402305,-4.084600,-4.234331,-4.033389,-4.231362,-3.602060,...,-3.795880,-3.638272,-4.903090,-3.920819,-4.549751,-4.037631,-5.269218,-2.966576,-4.649752,-4.131944


time: 12.1 ms (started: 2023-07-11 10:49:30 -05:00)


In [7]:
# correlation matrix

list_groups_subgroups_t_corr = correlation_global(list_groups_subgroups_t, "pearson")
list_groups_subgroups_t_corr[2][1].head()

100%|██████████| 3/3 [00:00<00:00, 11.50it/s]


,59.01425,61.98612,71.01184,74.00508,74.02387,78.95773,82.02924,83.06079,84.00671,84.04674,...,1155.95496,1158.45068,1166.35645,1166.93823,1170.16260,1177.92834,1178.40808,1180.43262,1185.20862,1197.05481
59.01425,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,...,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0
61.98612,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,...,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0
71.01184,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,...,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0
74.00508,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0
74.02387,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,...,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0


time: 281 ms (started: 2023-07-11 10:49:30 -05:00)


In [9]:
# build graph

list_groups_subgroups_t_corr_g = build_graph_weight_global(exp, list_groups_subgroups_t_corr, groups_id, subgroups_id, threshold=0.5)
list_groups_subgroups_t_corr_g[2][0].head()

100%|██████████| 3/3 [04:50<00:00, 96.69s/it]


,source,target,weight
1,59.01425,61.98612,1.0
2,59.01425,71.01184,-1.0
3,59.01425,74.00508,1.0
4,59.01425,74.02387,-1.0
5,59.01425,78.95773,1.0


time: 4min 50s (started: 2023-07-11 10:49:31 -05:00)


In [10]:
# create dataset - nodes/edge data for DGL framework

create_graph_data(exp, groups_id, subgroups_id)

  0%|          | 0/3 [00:00<?, ?it/s]

### Dynamic graph to Static graph

In [ ]:
create_graph_data_other(exp, groups_id, subgroups_id, option="dyn")

100%|██████████| 3/3 [00:47<00:00, 15.68s/it]]


: 

: 

In [ ]:
df_nodes = pd.read_csv("output/{}/preprocessing/graphs_data/nodes_data_{}_dyn.csv".format(exp, groups_id[0]))
df_nodes.head()

,idx,degree,id
0,0,11,A59.0049
1,1,26,A274.0123
2,2,65,A277.0867
3,3,61,A369.1213
4,4,70,A369.1541


time: 16 ms (started: 2023-06-08 08:07:29 -05:00)


In [ ]:
df_edges = pd.read_csv("output/{}/preprocessing/graphs_data/edges_data_{}_dyn.csv".format(exp, groups_id[0]))
print(df_edges.shape)
df_edges.head()

(12257101, 3)


,source,target,weight
0,0,1,0.629097
1,0,2,0.526495
2,0,3,0.530122
3,0,4,0.524363
4,0,5,0.504517


time: 2.15 s (started: 2023-06-08 08:07:29 -05:00)


In [ ]:
create_graph_data_other(groups_id, subgroups_id, option="str")

100%|██████████| 3/3 [07:14<00:00, 144.91s/it]

time: 7min 14s (started: 2023-06-08 08:33:36 -05:00)


In [ ]:
df_nodes = pd.read_csv("output/{}/preprocessing/graphs_data/nodes_data_{}_str.csv".format(exp, groups_id[0]))
df_nodes.head()

In [ ]:
df_edges = pd.read_csv("output/{}/preprocessing/graphs_data/edges_data_{}_str.csv".format(exp, groups_id[0]))
print(df_edges.shape)
df_edges.head()